# Introduction to TreeTrace module

**NOTE: This notebooks deals with both standard tracing nodes use and some low-level aspects of nodes and their storage.**

TreeTrace nodes are a framework for logging, tracing, result storage, and visualization of nested computations and actor interactions.
They have been designed with large textual and structured (e.g. JSON) inputs and outputs in mind, as well as generic and custom visualizations.

A `TracingNode` is an event of certain name, type (`kind`, with small differences in semantics for several built-in ones), start and end time, inputs,
result or error (exception), tags, and any child nodes - forming a rooted tree. Tags allow for tracing node filtering and search. Inputs and result support structured data and visualizations (see below).

## Storage

Currently, TracingNodes are stored as JSON files (one JSON file for every designated stored root node), but in the future we plan to also support DB storage.

## Visualization and UI

The framework also contains a web-based browser for the browsing tracing nodes - both directly in a jupyter notebook and in a separate web-browser window.

Tracing nodes and their visualization interact well with dataclasses (and JSON-like data in general) but also support custom visualizations: bitmap images, HTML, SVG (more can be added).

The current browser also shows still running TracingNodes (manual reloads required for refresh).

## In InterLab

Tracing nodes are an independent part of InterLab - you can use them in other projects (e.g. for the tracing browser), or use interlab without tracing nodes (some TracingNodes are still created but not stored by default, and should pose a trivial performance penalty comparable to logging).


In [1]:
from dataclasses import dataclass
import json
import matplotlib.pyplot as plt
from treetrace import TracingNode, with_trace, current_tracing_node, Tag, FileStorage
from treetrace.ext.pyplot import capture_figure

def show(obj):
    print(json.dumps(obj, indent=2))

## Basic tracing node examples

### Internals

In [2]:
# Tracing node with input and output
with TracingNode("MyTracingNode", inputs={"a": 10, "b": 20}) as node:
    node.set_result("Lorem ipsum ...")

node.display()

In [3]:
# How it looks as JSON
show(node.to_dict())

{
  "_type": "TracingNode",
  "name": "MyTracingNode",
  "uid": "2024-01-20T20-22-57-MyTracingNode-oxZRTf",
  "version": "3.0",
  "result": "Lorem ipsum ...",
  "inputs": {
    "a": 10,
    "b": 20
  },
  "start_time": "2024-01-20T20:22:57.824213",
  "end_time": "2024-01-20T20:22:57.824243"
}


In [4]:
# Tracing node nesting
with TracingNode("MyTracingNode") as node:
    with TracingNode("ChildNode1") as n1:
        n1.set_result("result")
    with TracingNode("ChildNode2") as n2:
        n2.set_result("result2")
    node.set_result("result")
node.display()

### Functions

In [5]:
# Tracing decorator

@with_trace
def my_function(a, b):
    return a + b

with TracingNode("root") as node:
    my_function(10, 20)
    
node.display()

### Exceptions

In [6]:
# Error handling

@with_trace
def my_function(a, b):
    raise Exception("Oops")

try:
    with TracingNode("root") as failing_node:
        my_function(10, 20)
except:
    pass

failing_node.display()

In [7]:
# note: errors must still be handled!

# This is expected to throw an error - note the tracing node would still be stored if you set up storage here
with TracingNode("root") as node:
    my_function(1, 2)

Exception: Oops

In [8]:
node.display()

### Dataclasses

In [9]:
# Dataclasses are serialized for logging inside tracing nodes

@dataclass
class Person:
    name: str
    age: int
    
@with_trace
def say_hi(person):
    return f"Hi {person.name}!"

with TracingNode("root") as node:
    person = Person("Alice", 21)
    say_hi(person)

node.display()

In [10]:
# same with dataclass outputs
@with_trace
def give_birth(name):
    return Person(age=0, name=name)

with TracingNode("root") as node:
    give_birth("Alice")

node.display()

### Tags

In [11]:
# Tags

with TracingNode("root", tags=["tag1", "tag2"]) as node:
    node.add_tag("exp1")  # Add to a tracing node dynamically
    node.add_tag(Tag("success!", color="lightgreen"))  # Add colored tag

node.display()

### Images and Plots

In [12]:
with TracingNode("root") as root:
    with TracingNode("first", meta={"color": "lightgreen"}):
        pass
    with TracingNode("second", meta={"color": "lightblue"}):
        pass        

    fig, ax = plt.subplots()
    
    fruits = ['apple', 'blueberry', 'cherry', 'orange']
    counts = [40, 100, 30, 55]
    bar_labels = ['red', 'blue', '_red', 'orange']
    bar_colors = ['tab:red', 'tab:blue', 'tab:red', 'tab:orange']
    
    ax.bar(fruits, counts, label=bar_labels, color=bar_colors)
    
    with TracingNode("image demo", inputs={"my_chart": capture_figure(fig), "other_param": 42}) as root2:
        pass

root.display()

In [13]:
# get current tracing node

with TracingNode("root") as node:
    with TracingNode("child"):
        current_tracing_node().add_tag("tag1")

node.display()

### Events

In [14]:
# Events (instant node with immediate result)

with TracingNode("root") as node:
    node.add_event("Message to Alice", kind="message", data="Hi, Alice!")

node.display()

### Storage

In [15]:
# Register a tracing node into storage
!rm logs/treetrace_intro/*
storage=FileStorage("logs/treetrace_intro")
with TracingNode("root1", storage=storage):
    pass


In [16]:
# Manual writing a tracing node into storage

with TracingNode("root2", tags=["hello"]) as c:
    pass

storage.write_node(c)

In [17]:
# with block and storage; all root tracing nodes in storage block are written in storage

with storage:
    with TracingNode("root3"):  # <-- This node is root; it is automatcally written in storage
        with TracingNode("child"): # <-- This node is not root; not automatically written in storage
            pass

In [18]:
# Composing directory structure with tracing nodes:
# each node subtree marked with `directory=True` gets stored in a separete json rather than in the main json

with TracingNode("root4", storage=storage, directory=True) as root:
    with TracingNode("first child", directory=True):
        with TracingNode("a"):
            pass
        with TracingNode("b", tags=["hello"]):
            pass
    with TracingNode("second child", directory=True):
        with TracingNode("a"):
            pass
        with TracingNode("b"):
            pass

import pathlib
list(pathlib.Path(f"{storage.directory}/{root.uid}.ctx").rglob("*"))

[PosixPath('/home/gavento/proj/interlab/notebooks/logs/treetrace_intro/2024-01-20T20-23-09-root4-5HFiHl.ctx/_self.gz'),
 PosixPath('/home/gavento/proj/interlab/notebooks/logs/treetrace_intro/2024-01-20T20-23-09-root4-5HFiHl.ctx/2024-01-20T20-23-09-first_child-mN2p5P.ctx'),
 PosixPath('/home/gavento/proj/interlab/notebooks/logs/treetrace_intro/2024-01-20T20-23-09-root4-5HFiHl.ctx/2024-01-20T20-23-09-second_child-C98Yz5.ctx'),
 PosixPath('/home/gavento/proj/interlab/notebooks/logs/treetrace_intro/2024-01-20T20-23-09-root4-5HFiHl.ctx/2024-01-20T20-23-09-first_child-mN2p5P.ctx/_self.gz'),
 PosixPath('/home/gavento/proj/interlab/notebooks/logs/treetrace_intro/2024-01-20T20-23-09-root4-5HFiHl.ctx/2024-01-20T20-23-09-first_child-mN2p5P.ctx/2024-01-20T20-23-09-a-xKw4LV.full.gz'),
 PosixPath('/home/gavento/proj/interlab/notebooks/logs/treetrace_intro/2024-01-20T20-23-09-root4-5HFiHl.ctx/2024-01-20T20-23-09-first_child-mN2p5P.ctx/2024-01-20T20-23-09-a-xKw4LV.root.gz'),
 PosixPath('/home/gavento/

### Data browser

In [19]:
# Running data browser over storage

storage.start_server(port=5000)

[2024-01-20 20:23:09,251] INFO(treetrace.ui.storage_server): Started tracing UI server: <ServerHandle http://localhost:5000>


<ServerHandle http://localhost:5000>

In [20]:
# Show live content of storage
storage.live_display()

In [21]:
# Long running cell, you can observe it in browser in running state (needs to be refreshed manually)

with TracingNode("Long running", storage=storage):
    with TracingNode("Child1"):
        import time
        time.sleep(10)

In [22]:
# Read all stored (root) nodes

for node in storage.read_all_nodes():
    print(node.uid, node.name)

2024-01-20T20-23-09-root1-favMip root1
2024-01-20T20-23-09-root2-eDqFyT root2
2024-01-20T20-23-09-root3-WwoZmd root3
2024-01-20T20-23-09-Long_running-IDATaL Long running
2024-01-20T20-23-09-root4-5HFiHl root4


In [23]:
# Recursively search for specific tracing nodes

for node in storage.find_nodes(lambda ctx: ctx.has_tag_name("hello")):
    print(node.uid, node.name)

In [24]:
# Read a tracing node by uid

node = storage.read_node(root.uid)
print(node.uid, node.name)


# Search in specific tracing nodes

node.find_nodes(lambda x: x.name == "a")

2024-01-20T20-23-09-root4-5HFiHl root4
